In [1]:
import pandas as pd
import numpy as np
import os
import csv
from tqdm import tqdm
import torch

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from one_peace.models import from_pretrained

In [2]:
al_model = torch.load("/workspace/jaeyoung/checkpoints/one_peace/esd_mmstts_al_0917/checkpoint_best.pt")
vl_model = torch.load("/workspace/jaeyoung/checkpoints/one_peace/mmtts_vl_0907/checkpoint_best.pt")

In [ ]:
al_model = from_pretrained(
    model_name_or_path="/workspace/jaeyoung/checkpoints/one_peace/esd_mmstts_al_0917/checkpoint_best.pt",
    model_type="one_peace_classify",
    device='cuda',
    dtype="float16"
)

In [7]:
vl_model = from_pretrained(
    model_name_or_path="/workspace/jaeyoung/checkpoints/one_peace/mmtts_vl_0907/checkpoint_best.pt",
    model_type="one_peace_retrieval",
    device='cuda',
    dtype="float16"
)

KeyboardInterrupt: 

In [ ]:
labels = ['happy', 'angry', 'sad', 'neutral', 'surprised', 'anxious']
img_list = ['/workspace/jaeyoung/datasets/mm-tts-dataset/video_image_save/M003_angry_level_1_001/0.jpg', '/workspace/jaeyoung/datasets/mm-tts-dataset/video_image_save/M003_fear_level_1_007/0.jpg', '/workspace/jaeyoung/datasets/mm-tts-dataset/video_image_save/M003_happy_level_2_001/1.jpg']
src_img = vl_model.process_image(img_list)

with torch.no_grad():
    img_logits = vl_model.extract_image_features(src_img)
    predicted_label_ids = img_logits.argmax(1).cpu().tolist()
print(predicted_label_ids)

# for img, pid in zip(img_list, predicted_label_ids):
#     predicted_label = 

NameError: name 'vl_model' is not defined

In [7]:
al_model.keys()

dict_keys(['args', 'cfg', 'model', 'criterion', 'optimizer_history', 'task_state', 'extra_state'])

In [11]:
al_model['model']

OrderedDict([('logit_scale', tensor(2.6562)),
             ('encoder_wrapper.text_adapter.cls_embedding',
              tensor([[[-0.0157, -0.0098, -0.0216,  ..., -0.0126,  0.0059,  0.0281]]])),
             ('encoder_wrapper.text_adapter.rp_bucket',
              tensor([[513, 511, 511,  ..., 511, 511, 511],
                      [512, 255, 254,  ...,   0,   0,   0],
                      [512, 256, 255,  ...,   0,   0,   0],
                      ...,
                      [512, 510, 510,  ..., 255, 254, 253],
                      [512, 510, 510,  ..., 256, 255, 254],
                      [512, 510, 510,  ..., 257, 256, 255]])),
             ('encoder_wrapper.text_adapter.embed_tokens.weight',
              tensor([[-0.0011,  0.0056, -0.0155,  ...,  0.0194,  0.0011,  0.0186],
                      [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.0036, -0.0172, -0.0283,  ..., -0.0082, -0.0129,  0.0104],
                      ...,
              

In [13]:
al_model['model'].keys()

odict_keys(['logit_scale', 'encoder_wrapper.text_adapter.cls_embedding', 'encoder_wrapper.text_adapter.rp_bucket', 'encoder_wrapper.text_adapter.embed_tokens.weight', 'encoder_wrapper.text_adapter.embed_positions.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.0.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.1.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.2.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.3.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.4.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.5.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.6.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.7.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.8.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.9.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.10.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.11.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.12.weig

In [ ]:
vl_model['model'].keys()

odict_keys(['logit_scale', 'encoder_wrapper.text_adapter.cls_embedding', 'encoder_wrapper.text_adapter.rp_bucket', 'encoder_wrapper.text_adapter.embed_tokens.weight', 'encoder_wrapper.text_adapter.embed_positions.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.0.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.1.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.2.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.3.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.4.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.5.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.6.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.7.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.8.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.9.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.10.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.11.weight', 'encoder_wrapper.text_adapter.rel_pos_table_list.12.weig

In [6]:
vl_model.keys()

dict_keys(['args', 'cfg', 'model', 'criterion', 'optimizer_history', 'task_state', 'extra_state'])